# Programowanie funkcyjne w Pythonie

Od funkcji wyższego rzędu do `itertools` i narzędzi zwiększających deklaratywność.


## Cele
- przypomnieć kluczowe funkcje wbudowane (`map`, `filter`, `reduce`)
- zademonstrować moduły `functools`, `itertools`, `operator`
- łączyć funkcje w potoki transformacji danych


## Wbudowane funkcje wyższego rzędu
`map`, `filter`, `sorted` z `key` umożliwiają deklaratywne transformacje.


In [ ]:
from operator import itemgetter

players = [
    {"name": "Ada", "score": 120},
    {"name": "Bartek", "score": 85},
    {"name": "Celina", "score": 150},
]

scores = list(map(itemgetter("score"), players))
print(scores)

high_scores = list(filter(lambda player: player["score"] >= 100, players))
print(high_scores)

rank = sorted(players, key=itemgetter("score"), reverse=True)
print(rank)


## `itertools`: potoki danych
Biblioteka `itertools` dostarcza efektywnych iteratorów.


In [ ]:
from itertools import islice, cycle

colors = cycle(["red", "green", "blue"])
print([next(colors) for _ in range(5)])

numbers = (n for n in range(100) if n % 7 == 0)
print(list(islice(numbers, 5)))


## `reduce`, `partial` i kompozycja
`functools.reduce` i `partial` pomagają budować funkcje specjalistyczne.


In [1]:
from functools import reduce, partial
from operator import mul

factorial = lambda n: reduce(mul, range(1, n + 1), 1)
print(factorial(5))

def power(base, exponent):
    return base ** exponent

square = partial(power, exponent=2)
print(square(9))


120
81


**Podsumowanie:** Programowanie funkcyjne pozwala budować zwięzłe potoki transformacji.

**Pytanie kontrolne:** W jakich scenariuszach generator wygrywa z listą?


### 🧩 Zadanie 1
Przyjmij listę słowników z zakupami (`name`, `price`, `quantity`) i oblicz
łączną wartość paragonu oraz listę produktów droższych niż 100 zł, używając
funkcji funkcyjnych (`map`, `filter`, `reduce`).


In [ ]:
# Rozwiązanie Zadania 1
from functools import reduce

shopping = [
    {"name": "Monitor", "price": 799, "quantity": 1},
    {"name": "Kabel", "price": 30, "quantity": 2},
    {"name": "Płyta", "price": 120, "quantity": 3},
]

values = list(map(lambda item: item["price"] * item["quantity"], shopping))
receipt_total = reduce(lambda acc, value: acc + value, values, 0)
print("Suma paragonu:", receipt_total)

expensive = list(filter(lambda item: item["price"] > 100, shopping))
print(expensive)


### 🧩 Zadanie 2
Za pomocą `itertools.groupby` pogrupuj listę słów według długości.


In [ ]:
# Rozwiązanie Zadania 2
from itertools import groupby

words = ["ala", "python", "kod", "lista", "obserwator"]

for length, group in groupby(sorted(words, key=len), key=len):
    group_list = list(group)
    print(length, group_list)


## Paradygmat: czyste funkcje i transparentność referencyjna
Czysta funkcja to taka, która:
- dla tych samych argumentów zawsze zwraca ten sam wynik (deterministyczność),
- nie ma efektów ubocznych (nie modyfikuje stanu zewnętrznego, nie I/O),
- jest łatwa do testowania i komponowania.

Transparentność referencyjna oznacza, że wywołanie funkcji można zastąpić jej wynikiem
bez zmiany zachowania programu.

Przykład funkcji czystej vs nieczystej:


In [ ]:
from datetime import datetime

# Funkcja czysta: nie korzysta ze stanu zewnętrznego, tylko z argumentów
# i nie ma efektów ubocznych.
def add_tax(price: float, tax_rate: float) -> float:
    return round(price * (1 + tax_rate), 2)

# Funkcja nieczysta: polega na aktualnym czasie (stan zewnętrzny),
# więc ten sam kod może zwrócić inny wynik o innej godzinie.
def greeting(name: str) -> str:
    current_hour = datetime.now().hour  # efekt uboczny: odczyt systemowego czasu
    return ("Dzień dobry " if 6 <= current_hour < 18 else "Dobry wieczór ") + name

print(add_tax(100, 0.23))  # zawsze 123.0 dla tych samych argumentów
print(greeting("Ada"))    # zależy od godziny


## Niezmienność (immutability) i struktury danych
W FP częściej preferujemy niezmienność. W Pythonie listy i słowniki są mutowalne, 
ale krotki (tuple) i `frozenset` już nie. Niezmienność ogranicza błędy związane
z przypadkową modyfikacją danych.


In [ ]:
# Przykład: zamiast mutować listę, tworzymy nową wersję
nums = [1, 2, 3]
# Mutująca operacja (impure style):
nums.append(4)  # modyfikuje istniejącą strukturę
# Styl funkcyjny: tworzymy nową listę na podstawie starej (bez modyfikacji oryginału)
nums2 = nums + [5]  # nowy obiekt
print("oryginał:", nums)
print("nowa    :", nums2)


## Funkcje wyższego rzędu i domknięcia (closures)
Funkcja wyższego rzędu (HOF) przyjmuje inne funkcje lub zwraca funkcję. Domknięcie
zapamiętuje otoczenie zmiennych w chwili definicji.


In [ ]:
from typing import Callable

# HOF, który zwraca funkcję skalującą
# Wewnętrzna funkcja scale korzysta ze zmiennej factor ze scope zewnętrznego (closure)
def make_scaler(factor: float) -> Callable[[float], float]:
    def scale(x: float) -> float:
        return x * factor  # użycie wartości spoza lokalnego scope => domknięcie
    return scale

by2 = make_scaler(2)
by10 = make_scaler(10)
print(by2(3), by10(3))  # 6 30


## `map`, `filter` i porównanie z list comprehensions
`map` i `filter` działają leniwie (zwroty iteratorów), a list comprehensions są bardzo czytelne
i idiomatyczne w Pythonie. Często LC są preferowane dla prostych przypadków.


In [ ]:
# Transformacja: kwadraty liczb parzystych
nums = list(range(10))
# map + filter (FP):
res_fp = list(map(lambda x: x * x, filter(lambda x: x % 2 == 0, nums)))
# List comprehension (idiomatyczne w Py):
res_lc = [x * x for x in nums if x % 2 == 0]
print(res_fp, res_lc)


## `functools`: `reduce`, `partial`, `lru_cache`
`reduce` składa elementy do jednej wartości; `partial` pozwala tworzyć wyspecjalizowane
funkcje; `lru_cache` dodaje memoizację (zapamiętywanie wyników) do funkcji czystych.


In [ ]:
from functools import reduce, partial, lru_cache
from operator import mul

# reduce: iloczyn elementów (z wartością początkową 1, aby pusta lista dała 1)
product = reduce(mul, [2, 3, 4], 1)
print("product:", product)

# partial: selektor z góry ustalonym polem
from operator import itemgetter
get_price = itemgetter("price")

# Tworzymy funkcję rabatu 10% poprzez cząstkowe związanie argumentu tax_rate
apply_vat_23 = partial(add_tax, tax_rate=0.23)
print(apply_vat_23(100))

# lru_cache: memoizacja kosztownej funkcji czystej
@lru_cache(maxsize=128)
def fib(n: int) -> int:
    # Prosty (nieefektywny) fib z memoizacją – na potrzeby demonstracji
    if n < 2:
        return n
    return fib(n - 1) + fib(n - 2)

print([fib(i) for i in range(10)])


## `itertools`: więcej przykładów
Kombinowanie iteratorów bez tworzenia pośrednich list oszczędza pamięć i CPU.


In [ ]:
from itertools import chain, accumulate, groupby, islice
from operator import add

# chain: łączenie wielu sekwencji w jeden strumień
stream = chain(range(3), "ab", [10, 11])
print(list(stream))

# accumulate: akumulator prefiksowy (tu: suma prefiksowa)
print(list(accumulate([1, 2, 3, 4], func=add)))  # [1, 3, 6, 10]

# groupby: grupowanie po kluczu – uwaga: wymaga posortowania po kluczu!
words = ["ala", "python", "kod", "lista", "obserwator"]
for length, group in groupby(sorted(words, key=len), key=len):
    print(length, list(group))


## `operator`: szybkie funkcje jednoargumentowe i wieloargumentowe
Moduł `operator` udostępnia gotowe funkcje, które często są czytelniejsze niż lambdy.


In [ ]:
from operator import attrgetter, itemgetter, methodcaller

people = [
    {"name": "Ada", "age": 34},
    {"name": "Bartek", "age": 28},
    {"name": "Celina", "age": 34},
]

# itemgetter po kluczu
print(sorted(people, key=itemgetter("age", "name")))

class User:
    def __init__(self, name: str):
        self.name = name
    def greet(self) -> str:
        return f"Hello {self.name}"

users = [User("Zoe"), User("Adam")]
print(list(map(attrgetter("name"), users)))  # ['Zoe', 'Adam']
print(list(map(methodcaller("greet"), users)))  # wywołuje metodę na każdym obiekcie


## Kompozycja funkcji i potoki (pipelines)
W FP łączymy małe funkcje w większe potoki. W Pythonie możemy zaimplementować prosty
kompozytor.


In [ ]:
from functools import reduce
from typing import Iterable, Any, Callable

# prosta kompozycja: compose(f, g)(x) == f(g(x))
def compose(*funcs: Callable[[Any], Any]) -> Callable[[Any], Any]:
    def composed(arg: Any) -> Any:
        return reduce(lambda acc, f: f(acc), reversed(funcs), arg)
    return composed

strip = str.strip
lower = str.lower
remove_hash = lambda s: s.replace("#", "")  # funkcja anonimowa (lambda) do prostego czyszczenia

clean_hashtag = compose(remove_hash, strip, lower)  # najpierw lower, potem strip, na końcu remove_hash
print(clean_hashtag("  #Python  "))  # 'python'


## Obsługa błędów w stylu funkcyjnym: walidacja jako funkcje
Zamiast `try/except` w wielu miejscach, można budować potoki walidatorów.


In [ ]:
from typing import Tuple

Validation = Tuple[bool, str]

def not_empty(s: str) -> Validation:
    return (len(s.strip()) > 0, "Pole nie może być puste")

def min_length(n: int) -> Callable[[str], Validation]:
    def check(s: str) -> Validation:
        return (len(s) >= n, f"Minimalna długość to {n}")
    return check

validators = [not_empty, min_length(3)]

def validate(s: str, validators) -> Validation:
    # Zwraca (True, "") jeśli wszystko ok lub (False, message) dla pierwszego błędu
    for v in validators:
        ok, msg = v(s)
        if not ok:
            return False, msg
    return True, ""

print(validate("", validators))        # (False, 'Pole nie może być puste')
print(validate("ab", validators))      # (False, 'Minimalna długość to 3')
print(validate("python", validators))  # (True, '')


## Efekty uboczne a FP: kiedy są w porządku?
FP preferuje brak efektów ubocznych, ale w praktyce musimy robić I/O. Kluczowe jest
izolowanie efektów na krawędziach systemu i utrzymywanie rdzenia obliczeń jako czystych funkcji.


### 🧩 Zadanie 3
Zbuduj potok czyszczący listę wpisów tekstowych: usuń `None`, przytnij spacje,
usuwaj puste wpisy i zwróć posortowaną (case-insensitive) listę unikatowych wartości.
Użyj `filter`, `map`, `str.strip`, `str.lower`, `set` i `sorted`.


In [ ]:
# Rozwiązanie Zadania 3
entries = ["  Python", None, "", " PYTHON ", "fp", "Fp  ", "  "]

cleaned = sorted(
    set(
        map(
            str.lower,  # zamiana na małe litery
            filter(
                None,  # odfiltruj False'y: None, "", "  " po strip -> patrz krok niżej
                map(
                    str.strip,  # przytnij spacje
                    filter(lambda x: x is not None, entries),  # usuń None
                ),
            ),
        )
    )
)
print(cleaned)  # ['fp', 'python']


### 🧩 Zadanie 4
Wykorzystaj `lru_cache`, aby przyspieszyć funkcję obliczającą koszty dostawy, która jest
funkcją czystą zależną od (wagi, dystansu). Porównaj czasy dla wielokrotnych wywołań
z tymi samymi parametrami.


In [ ]:
# Rozwiązanie Zadania 4
import time
from functools import lru_cache

@lru_cache(maxsize=None)
def shipping_cost(weight: float, distance: int) -> float:
    # Udajemy kosztowną funkcję – np. symulacja złożonego algorytmu
    time.sleep(0.05)  # opóźnienie
    base = 5.0
    return base + 0.4 * weight + 0.02 * distance

start = time.perf_counter()
for _ in range(5):
    shipping_cost(10, 100)  # te same argumenty
print("z cache:", time.perf_counter() - start)

shipping_cost.cache_clear()
start = time.perf_counter()
for _ in range(5):
    shipping_cost.__wrapped__(10, 100)  # wywołanie bez cache dla porównania
print("bez cache:", time.perf_counter() - start)


### 🧩 Zadanie 5
Porównaj dwa style: `map`/`filter` vs list/dict comprehensions, implementując:
- słownik {litera: liczba_wystąpień} dla liter w tekście (tylko alfanumeryczne, małe litery),
- uporządkowaną listę par (litera, licznik) malejąco po liczniku.


In [ ]:
# Rozwiązanie Zadania 5
from collections import Counter
text = "Functional Programming in Python 3.12!!!"
letters = [ch.lower() for ch in text if ch.isalnum()]  # idiomatycznie
counts = Counter(letters)
result = sorted(counts.items(), key=lambda kv: kv[1], reverse=True)
print(result)


**Podsumowanie rozszerzenia:**
- Czeste preferowanie czystych funkcji ułatwia testowanie i kompozycję.
- `functools` i `itertools` dostarczają potężnych klocków do budowy potoków.
- Immutability zmniejsza klasę błędów związanych ze współdzieleniem stanu.
- W Pythonie pamiętaj o idiomach (comprehensions), jednocześnie korzystając z idei FP tam, gdzie to pomaga.
